In [1]:
# import os
# # Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# # For example:
# # spark_version = 'spark-3.3.0'
# spark_version = 'spark-3.3.0'
# os.environ['SPARK_VERSION']=spark_version

In [82]:
import pandas as pd

structures_df = pd.read_csv('https://purifai.s3.us-west-1.amazonaws.com/data/structures.csv')
outcomes_df = pd.read_csv('https://purifai.s3.us-west-1.amazonaws.com/data/outcomes.csv')

outcomes_df.head()

,SAMPLE_ID,STRUCTURE_ID,PREFERRED_LCMS_METHOD,SPE_METHOD,METHOD,SPE_SUCCESSFUL,CRASHED_OUT,SAMPLE_STATUS,SAMPLE_CURRENT_STATUS,TERMINATION_CAUSE,TERMINATION_STEP,TERMINATION_DETAILS,REACTION_SCALE_(mmol),SELECTED_FRACTIONS,VOLUME_COLLECTED_(mL),TOTAL_FRACTIONS_COLLECTED,RECOVERED_SAMPLE_DRY_MASS_(mg),PERCENT_YIELD,%_PURITY_(BY_LCMS),PURIFICATION_COMMENTS
0,MTA0ST2022-051-001,MTA0ST2022-051-001,Xbridge HpH,MCX,MCX/Xbridge HpH,NaN,NaN,Failed,Terminated,Challenges of Sample/Chemistry,Synthesis (SAP1/2),No conversion (SM is major peak),0.1,NaN,NaN,NaN,NaN,0.48,NaN,NaN
1,MTA0ST2022-051-001_S2,MTA0ST2022-051-001,Xbridge HpH,HLB,HLB/Xbridge HpH,TRUE,NaN,Complete,Completed & Stored,NaN,NaN,NaN,0.1,F3L/F4S,10.17,2.0,17.6,NaN,0.99,NaN
2,MTA0ST2022-051-002,MTA0ST2022-051-002,NaN,NaN,NaN,NaN,NaN,Failed,Terminated,Challenges of Sample/Chemistry,Synthesis (SAP1/2),"Insufficient COI detected, below threshold,No ...",0.1,NaN,NaN,NaN,NaN,0.11,NaN,NaN
3,MTA0ST2022-051-003_G1,MTA0ST2022-051-003,Xbridge HpH,MCX,MCX/Xbridge HpH,TRUE,NaN,Complete,Completed & Stored,NaN,NaN,NaN,0.1,F3L/F1S,19.46,2.0,3.9,0.85,0.98,NaN
4,MTA0ST2022-051-003_G2,MTA0ST2022-051-003,Xbridge HpH,MCX,MCX/Xbridge HpH,NaN,NaN,Failed,Terminated,Challenges of Sample/Chemistry,Synthesis (SAP1/2),No conversion (SM is major peak),0.1,F3L/F1S,9.78,2.0,3.0,0.41,NaN,";;;""2nd group; no COI observed"";"


In [83]:
#cleaning structures dataframe
structures_df = structures_df.drop_duplicates(subset=['STRUCTURE_ID'], keep='first')

In [84]:
#cleaning outcomes dataframe
# Review data types
dict(outcomes_df.dtypes)

outcomes_df = outcomes_df[outcomes_df['STRUCTURE_ID'].notna()]
outcomes_df = outcomes_df.drop(columns="PURIFICATION_COMMENTS")
outcomes_df.head()

,SAMPLE_ID,STRUCTURE_ID,PREFERRED_LCMS_METHOD,SPE_METHOD,METHOD,SPE_SUCCESSFUL,CRASHED_OUT,SAMPLE_STATUS,SAMPLE_CURRENT_STATUS,TERMINATION_CAUSE,TERMINATION_STEP,TERMINATION_DETAILS,REACTION_SCALE_(mmol),SELECTED_FRACTIONS,VOLUME_COLLECTED_(mL),TOTAL_FRACTIONS_COLLECTED,RECOVERED_SAMPLE_DRY_MASS_(mg),PERCENT_YIELD,%_PURITY_(BY_LCMS)
0,MTA0ST2022-051-001,MTA0ST2022-051-001,Xbridge HpH,MCX,MCX/Xbridge HpH,NaN,NaN,Failed,Terminated,Challenges of Sample/Chemistry,Synthesis (SAP1/2),No conversion (SM is major peak),0.1,NaN,NaN,NaN,NaN,0.48,NaN
1,MTA0ST2022-051-001_S2,MTA0ST2022-051-001,Xbridge HpH,HLB,HLB/Xbridge HpH,TRUE,NaN,Complete,Completed & Stored,NaN,NaN,NaN,0.1,F3L/F4S,10.17,2.0,17.6,NaN,0.99
2,MTA0ST2022-051-002,MTA0ST2022-051-002,NaN,NaN,NaN,NaN,NaN,Failed,Terminated,Challenges of Sample/Chemistry,Synthesis (SAP1/2),"Insufficient COI detected, below threshold,No ...",0.1,NaN,NaN,NaN,NaN,0.11,NaN
3,MTA0ST2022-051-003_G1,MTA0ST2022-051-003,Xbridge HpH,MCX,MCX/Xbridge HpH,TRUE,NaN,Complete,Completed & Stored,NaN,NaN,NaN,0.1,F3L/F1S,19.46,2.0,3.9,0.85,0.98
4,MTA0ST2022-051-003_G2,MTA0ST2022-051-003,Xbridge HpH,MCX,MCX/Xbridge HpH,NaN,NaN,Failed,Terminated,Challenges of Sample/Chemistry,Synthesis (SAP1/2),No conversion (SM is major peak),0.1,F3L/F1S,9.78,2.0,3.0,0.41,NaN


In [87]:
# Convert percentage columns to numerical
## SOME OF THE "PERCENT_YIELD" COLUMNS HAVE #VALUE! IN THEM, WHICH NEEDS TO BE RESOLVED BEFORE THIS CONVERSION CAN BE COMPLETED
outcomes_df['PERCENT_YIELD'] = outcomes_df['PERCENT_YIELD'].replace(['#VALUE!'],'0')
# outcomes_df["PERCENT_YIELD"] = outcomes_df["PERCENT_YIELD"].str.replace("%", "")
# outcomes_df["PERCENT_YIELD"] = outcomes_df["PERCENT_YIELD"].astype("float") / 100

# outcomes_df = outcomes_df.rename(columns = {"Unnamed: 20": "EXTRA_COMMENTS"})
outcomes_df["%_PURITY_(BY_LCMS)"] = outcomes_df["%_PURITY_(BY_LCMS)"].str.replace("%", "")
outcomes_df["%_PURITY_(BY_LCMS)"] = outcomes_df["%_PURITY_(BY_LCMS)"].str.replace("?", "")
outcomes_df["%_PURITY_(BY_LCMS)"] = outcomes_df["%_PURITY_(BY_LCMS)"].astype("float") / 100


/Users/lukeperrin/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if __name__ == "__main__":


In [71]:
print(outcomes_df.head(10))

               SAMPLE_ID        STRUCTURE_ID PREFERRED_LCMS_METHOD SPE_METHOD  \
0     MTA0ST2022-051-001  MTA0ST2022-051-001           Xbridge HpH        MCX   
1  MTA0ST2022-051-001_S2  MTA0ST2022-051-001           Xbridge HpH        HLB   
2     MTA0ST2022-051-002  MTA0ST2022-051-002                   NaN        NaN   
3  MTA0ST2022-051-003_G1  MTA0ST2022-051-003           Xbridge HpH        MCX   
4  MTA0ST2022-051-003_G2  MTA0ST2022-051-003           Xbridge HpH        MCX   
5     MTA0ST2022-051-004  MTA0ST2022-051-004            Gemini LpH        MCX   
6     MTA0ST2022-051-005  MTA0ST2022-051-005            Gemini LpH        MCX   
7     MTA0ST2022-051-006  MTA0ST2022-051-006           Xbridge HpH        MCX   
8     MTA0ST2022-051-007  MTA0ST2022-051-007           Xbridge HpH        MCX   
9     MTA0ST2022-051-008  MTA0ST2022-051-008                   NaN        NaN   

            METHOD SPE_SUCCESSFUL  CRASHED_OUT SAMPLE_STATUS  \
0  MCX/Xbridge HpH            NaN          N

In [88]:
SPE_analysis_df = pd.merge(outcomes_df, structures_df, on=["STRUCTURE_ID"])
SPE_analysis_df = SPE_analysis_df[SPE_analysis_df['SPE_SUCCESSFUL'] == "TRUE"]
SPE_analysis_df.head()

,SAMPLE_ID,STRUCTURE_ID,PREFERRED_LCMS_METHOD,SPE_METHOD,METHOD,SPE_SUCCESSFUL,CRASHED_OUT,SAMPLE_STATUS,SAMPLE_CURRENT_STATUS,TERMINATION_CAUSE,...,NumAromaticHeterocycles,NumAromaticRings,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount
1,MTA0ST2022-051-001_S2,MTA0ST2022-051-001,Xbridge HpH,HLB,HLB/Xbridge HpH,TRUE,NaN,Complete,Completed & Stored,NaN,...,1,2,4,2,9,2,0,1,1,3
3,MTA0ST2022-051-003_G1,MTA0ST2022-051-003,Xbridge HpH,MCX,MCX/Xbridge HpH,TRUE,NaN,Complete,Completed & Stored,NaN,...,1,2,4,1,8,2,0,1,1,3
5,MTA0ST2022-051-004,MTA0ST2022-051-004,Gemini LpH,MCX,MCX/Gemini LpH,TRUE,NaN,Complete,Completed & Stored,NaN,...,0,2,3,1,6,2,0,1,1,3
6,MTA0ST2022-051-005,MTA0ST2022-051-005,Gemini LpH,MCX,MCX/Gemini LpH,TRUE,NaN,Complete,Completed & Stored,NaN,...,1,2,2,1,6,2,0,1,1,3
7,MTA0ST2022-051-006,MTA0ST2022-051-006,Xbridge HpH,MCX,MCX/Xbridge HpH,TRUE,NaN,Complete,Completed & Stored,NaN,...,1,3,2,1,6,2,0,1,1,4


In [108]:
outcomes_df.to_csv('./cleaned_data/cleaned-outcomes.csv', index=False)
structures_df.to_csv('./cleaned_data/cleaned-structures.csv', index=False)
SPE_analysis_df.to_csv('./cleaned_data/SPE-analysis.csv', index=False)